In [2]:
import numpy as np
import cv2
import pydicom as dicom #전처리에서는 쓰지 않음
import SimpleITK as sitk
from math import sqrt

In [3]:
filename = './1.3.12.2.1107.5.2.19.45117.2018090613503685637465765.dcm'
images = sitk.ReadImage(filename)
images_array = sitk.GetArrayFromImage(images).astype('float32')

일반적으로 한 이미지를 구성하고 있는 pixel 값의 범위는 기본적으로 0부터 255, 0부터 2^8 범위를 가지고 있어 8bit depth라고 할 수 있음\
하지만 DICOM 파일의 경우 12 bit depth, 16 bit depth로 구성되어 있는 것도 많음.\
따라서 DICOM 파일에서 max 값과 min 값을 각각 계산하고, 모든 값을 0~1 사이로 정규화 해주기\
이후 0~1 사이로 정규화 한 값들에 255를 곱해 8 bit depth로 바꿔주기


In [4]:
img = np.squeeze(images_array)
copy_img = img.copy()
min=np.min(copy_img)
max=np.max(copy_img)
min, max

(0.0, 2039.0)

In [5]:
copy_img = (copy_img - min)/(max-min)
copy_img *= 2**8 -1
copy_img = copy_img.astype(np.uint8)

copy_img = np.expand_dims(copy_img, axis = -1)
copy_img = cv2.cvtColor(copy_img, cv2.COLOR_GRAY2RGB)

In [6]:
np.min(copy_img), np.max(copy_img)

(0, 255)

In [8]:
sobel_x = cv2.Sobel(copy_img, -1, 1, 0)
sobel_y = cv2.Sobel(copy_img, -1, 0, 1)
sobel = sobel_x + sobel_y
final = copy_img - cv2.bitwise_not(sobel)
print(sobel.shape)
#sobel = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5,0)

cv2.imshow('sample image dicom',copy_img)
cv2.imshow('sobel_x', sobel_x)
cv2.imshow('sobel_y', sobel_y)
cv2.imshow('sobel', sobel)
cv2.imshow('final', final)
cv2.waitKey()

cv2.destroyAllWindows()

(384, 384, 3)


In [12]:
#cv2.HoghCircles 안에 이미 Canny edge detection이 내부적으로 들어가 있어서 굳이 필요 없을 듯
canny = cv2.Canny(sobel, 50, 150)
canny = cv2.cvtColor(canny, cv2.COLOR_GRAY2RGB)
print(canny.shape)

cv2.imshow('canny edge', canny)
cv2.waitKey()

cv2.destroyAllWindows()


(384, 384, 3)


HoughCircles
- 인자 1 - 그레이 이미지
- 인자 2 - cv2.HOUGH_GRADIENT : 허프 번환을 이용해 원 찾기
- 인자 3 - 원본 이미지와 허프 변환 카운팅 결과 이미지의 비
- 인자 4 - 찾은 원들의 중심 간 최고 거리. 중심 간의 거리아 이 값보다 작으면 나중에 찾은 원 무시. 값을 적절하게 조절할 필요 있음
- 인자 5 - param1 : Canny()의 maxVal 값
- 인자 6 - param2 : 원으로 판단하는 허프 변환 카운팅 갓. 작을 수록 많은 원, 클 수록 적은 원

In [56]:
circle_img = cv2.cvtColor(copy_img, cv2.COLOR_RGB2GRAY)
print(circle_img)
circles = cv2.HoughCircles(circle_img,cv2.HOUGH_GRADIENT,1,10,param1=240,param2=13,minRadius=0,maxRadius=11) 
circles = np.uint16(np.around(circles))
circle_img = cv2.cvtColor(circle_img, cv2.COLOR_GRAY2RGB)
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(circle_img,(i[0],i[1]),i[2],(0,0,255),1)
    # draw the center of the circle
    #cv2.circle(circle_img,(i[0],i[1]),2,(0,0,255),3)

cv2.imshow('detected circles',circle_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [55]:
circle_img = cv2.cvtColor(copy_img, cv2.COLOR_RGB2GRAY)
circles = cv2.HoughCircles(circle_img,cv2.HOUGH_GRADIENT,1,10,param1=240,param2=10,minRadius=0,maxRadius=10) 
circles = np.uint16(np.around(circles))
circle_img = cv2.cvtColor(circle_img, cv2.COLOR_GRAY2RGB)
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(circle_img,(i[0],i[1]),i[2],(0,0,255),1)
    # draw the center of the circle
    #cv2.circle(circle_img,(i[0],i[1]),2,(0,0,255),3)

cv2.imshow('detected circles',circle_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
ds=dicom.dcmread('./1.3.12.2.1107.5.2.19.45117.2018090613503685637465765.dcm')
dcm_sample=ds.pixel_array
cv2.imshow('sample image dicom',dcm_sample)

cv2.waitKey()
cv2.destroyAllWindows()